In [1]:
corpus = ['Казнить нельзя, помиловать. Нельзя наказывать.',
          'Казнить, нельзя помиловать. Нельзя освободить.',
          'Нельзя не помиловать.',
          'Обязательно освободить.']

In [2]:
corpus

['Казнить нельзя, помиловать. Нельзя наказывать.',
 'Казнить, нельзя помиловать. Нельзя освободить.',
 'Нельзя не помиловать.',
 'Обязательно освободить.']

In [3]:
import re

TOKEN_RE = re.compile(r'[\w\d]+')

def tokenize_text_simple_regex(txt, min_token_size=4):
  txt = txt.lower()
  all_tokens = TOKEN_RE.findall(txt)
  return [token for token in all_tokens if len(token) >= min_token_size]

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
  return [tokenizer(text, **tokenizer_kwargs) for text in texts]

In [4]:
corpus_tokenized = tokenize_corpus(corpus, min_token_size=0)

In [5]:
corpus_tokenized

[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'],
 ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'],
 ['нельзя', 'не', 'помиловать'],
 ['обязательно', 'освободить']]

In [62]:
import collections

import numpy as np

def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
  word_counts = collections.defaultdict(int)
  doc_n = 0

  # посчитать количество документов, в которых употребляется каждое слово,
  # а также общее количество документов
  for txt in tokenized_texts:
    doc_n += 1
    unique_text_tokens = set(txt)
    for token in unique_text_tokens:
      word_counts[token] += 1

  # убрать слишком редкие и слишком частые слова
  word_counts = {word: cnt for word, cnt in word_counts.items() if cnt >= min_count and cnt / doc_n <= max_doc_freq}

  # отсортировать слова по убыванию частоты
  sorted_word_counts = sorted(word_counts.items(),
                              reverse=True,
                              key=lambda pair: pair[1])

  # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
  if pad_word is not None:
    sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

  # если у нас по-прежнему слишком много слов, оставить только max_size самых частотных
  if len(word_counts) > max_size:
    sorted_word_counts = sorted_word_counts[:max_size]

  sorted_word_counts = [(word, cnt / doc_n) for word, cnt in sorted(sorted_word_counts, key=lambda pair: (pair[1], pair[0]))]

  # нумеруем слова
  word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

  # нормируем частоты слов
  word2freq = np.array([cnt for _, cnt in sorted_word_counts], dtype='float32')

  return word2id, word2freq

In [64]:
word2id, word2freq = build_vocabulary(corpus_tokenized, max_doc_freq=1.0, min_count=0)

In [65]:
print(*word2id.keys())
print(*word2freq)

наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75


In [143]:
import scipy.sparse

def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
  assert mode in {'tfidf', 'idf', 'tf', 'bin'}

  # считаем количество употреблений каждого слова в каждом документе
  result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
  for text_i, text in enumerate(tokenized_texts):
    for token in text:
      if token in word2id:
        result[text_i, word2id[token]] += 1

  # получаем бинарные вектора "встречается или нет"
  if mode == 'bin':
    result = (result > 0).astype('float32')

  # получаем вектора относительных частот слова в документе
  elif mode == 'tf':
    result = result.tocsr()
    result = result.multiply(1 / result.sum(1))

  # полностью убираем информацию о количестве употреблений слова в данном документе,
  # но оставляем информацию о частотности слова в корпусе в целом
  elif mode == 'idf':
    result = (result > 0).astype('float32').multiply(1 / word2freq)

  # учитываем всю информацию, которая у нас есть:
  # частоту слова в документе и частоту слова в корпусе
  elif mode == 'tfidf':
    result = result.tocsr()
    result = result.multiply(1 / result.sum(1)) # разделить каждую строку на ее длину
    result = result.multiply(1 / word2freq) # разделить каждый столбец на вес слова

  if scale:
    result = result.tocsr()
    result -= result.min()
    result /= (result.max() + 1e-6)
  
  return result.tocsr()

In [144]:
corpus_tf = vectorize_texts(corpus_tokenized, word2id, word2freq, 'tf').toarray()

In [145]:
corpus_tf

array([[0.3999992, 0.       , 0.       , 0.3999992, 0.       , 0.7999984,
        0.3999992],
       [0.       , 0.       , 0.       , 0.3999992, 0.3999992, 0.7999984,
        0.3999992],
       [0.       , 0.6666653, 0.       , 0.       , 0.       , 0.6666653,
        0.6666653],
       [0.       , 0.       , 0.999998 , 0.       , 0.999998 , 0.       ,
        0.       ]], dtype=float32)

In [146]:
corpus_tf.round(2)

array([[0.4 , 0.  , 0.  , 0.4 , 0.  , 0.8 , 0.4 ],
       [0.  , 0.  , 0.  , 0.4 , 0.4 , 0.8 , 0.4 ],
       [0.  , 0.67, 0.  , 0.  , 0.  , 0.67, 0.67],
       [0.  , 0.  , 1.  , 0.  , 1.  , 0.  , 0.  ]], dtype=float32)

In [147]:
corpus_idf = vectorize_texts(corpus_tokenized, word2id, word2freq, 'idf').toarray()

In [148]:
corpus_idf

array([[0.99999976, 0.        , 0.        , 0.49999988, 0.        ,
        0.33333325, 0.33333325],
       [0.        , 0.        , 0.        , 0.49999988, 0.49999988,
        0.33333325, 0.33333325],
       [0.        , 0.99999976, 0.        , 0.        , 0.        ,
        0.33333325, 0.33333325],
       [0.        , 0.        , 0.99999976, 0.        , 0.49999988,
        0.        , 0.        ]], dtype=float32)

In [149]:
import numpy as np

In [150]:
corpus_ltfidf = np.log(corpus_tf / 2 + 1) * corpus_idf

In [151]:
corpus_ltfidf

array([[0.18232116, 0.        , 0.        , 0.09116058, 0.        ,
        0.11215719, 0.06077372],
       [0.        , 0.        , 0.        , 0.09116058, 0.09116058,
        0.11215719, 0.06077372],
       [0.        , 0.2876815 , 0.        , 0.        , 0.        ,
        0.09589383, 0.09589383],
       [0.        , 0.        , 0.40546438, 0.        , 0.20273219,
        0.        , 0.        ]], dtype=float32)

In [152]:
m = corpus_ltfidf.mean(0)

In [153]:
m

array([0.04558029, 0.07192037, 0.1013661 , 0.04558029, 0.07347319,
       0.08005205, 0.05436032], dtype=float32)

In [154]:
s = corpus_ltfidf.std(0, ddof=1)

In [155]:
s

array([0.09116058, 0.14384075, 0.20273219, 0.05263159, 0.09629356,
       0.0539159 , 0.03984278], dtype=float32)

In [156]:
matrix = ((corpus_ltfidf - m) / s).round(2)

In [157]:
for i in matrix:
  print(*i)

1.5 -0.5 -0.5 0.87 -0.76 0.6 0.16
-0.5 -0.5 -0.5 0.87 0.18 0.6 0.16
-0.5 1.5 -0.5 -0.87 -0.76 0.29 1.04
-0.5 -0.5 1.5 -0.87 1.34 -1.48 -1.36
